In [198]:
import os
from zhipuai import ZhipuAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
# 获取API Key
os.environ["ZHIPUAI_API_KEY"] = "ZHIPUAI_API_KEY"
client = ZhipuAI() 

In [200]:
def generate_character_profiles(client, text):
    """使用ChatGLM系列模型中的glm-4模型生成角色人设"""
    template = """你是一位生成角色人设的专家。根据下面这段文本，你的任务是为该文本各用一段话来生成两个角色人设。

    文本：{text}

    以下是根据该文本生成的两个角色人设：
    """
    llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key="ZHIPUAI_API_KEY",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
    )

    prompt = PromptTemplate(template=template)
    chain = prompt | llm
    response = chain.invoke({text})

    #从响应中获取两位角色人设及它们的名字
    role1_info = response.content.split('\n\n')[0].split('\n')[1]
    role2_info = response.content.split('\n\n')[1].split('\n')[1]
    role1_name = response.content.split('\n\n')[0].split('\n')[0].split('：')[1]
    role2_name = response.content.split('\n\n')[1].split('\n')[0].split('：')[1]

    return role1_info,role2_info,role1_name,role2_name

In [202]:
def create_chat_completion(client, meta, messages):
    """使用CharacterGLM API生成对话回复"""
    response = client.chat.completions.create(
        model="charglm-3",
        meta=meta,
        messages=messages
    )
    return response.choices[0].message

In [204]:
def save_dialogue_to_file(dialogue, filename):
    """将对话记录保存到文件"""
    with open(filename, 'w', encoding='utf-8') as file:
        for message in dialogue:
            # 检查 message 是否包含 'role' 和 'content' 键
            if 'role' in message and 'content' in message:
                role = "用户" if message['role'] == 'user' else "机器人"
                content = message['content']
            else:
                # 如果 message 不是一个有效的字典，直接使用它
                role = "用户" if "user" in message else "机器人"
                content = message
            file.write(f"{role}: {content}\n")

In [206]:
#给定一段文本
text = """在一个安静的咖啡馆里，阳光透过窗户洒在木质的桌椅上，两位老朋友，杰克和艾米丽，坐在角落里的一张桌子旁，享受着午后的悠闲时光。
杰克轻轻搅动着杯中的拿铁，微笑着对艾米丽说：“你还记得我们第一次见面的情景吗？那时候你可是个害羞的小女孩。”
艾米丽轻轻笑了，她的眼睛里闪烁着回忆的光芒：“当然记得，你当时正拿着一本厚厚的书，看起来像是要吞噬整个世界。”
“哈哈，那本书确实让我着迷。”杰克回忆着，“那时候我正梦想着成为一名作家，没想到现在真的实现了。”
艾米丽点头，眼中满是赞赏：“你的书我都有读，每本都让我感动。你的文字有力量，能够触动人心。”
杰克谦虚地摇了摇头：“我只是把心里的故事写出来而已。倒是你，艾米丽，你的变化让我惊讶。记得你以前总是害怕站在人前，现在却能自信地领导一个团队。”
艾米丽轻轻抿了口咖啡，眼中闪过一丝坚定：“是的，我学会了面对自己的恐惧。你的文字给了我勇气，让我知道每个人都有自己的故事，值得被听见。”
两人的对话在咖啡的香气中缓缓流淌，时间仿佛在这一刻静止。他们谈论着过去，现在，以及未来的梦想，彼此的友谊在这些简单而又深刻的话语中愈发深厚。
咖啡馆里的音乐轻轻响起，杰克和艾米丽的对话继续，他们的故事也在继续，就像这永不结束的午后时光。
"""

In [208]:
#消息对话记录
messages= [
        {
            "role": "assistant",
            "content": "（旁白：在那个充满书香的咖啡馆里，杰克和艾米丽坐在窗边，享受着温暖的阳光和浓郁的咖啡香。他们的对话在轻松愉快的氛围中展开。）艾米丽，你还记得你第一次站在演讲台上的样子吗？那时候你紧张得手都在抖。"
          },
        {
            "role": "user",
            "content": "（轻笑着，眼中闪过一丝顽皮） 哦，杰克，你真是的。我那时候确实紧张，但你的鼓励让我迈出了那一步。"
          },
        {
            "role": "assistant",
            "content": "（点头，眼中满是骄傲） 是的，你做到了。你的演讲总是那么鼓舞人心，就像你带领团队一样。"
          },
        {
            "role": "user",
            "content": "（认真地） 我学到了很多，杰克。你的书教会了我如何用故事去影响别人，如何用文字去传递力量。"
          }
        ]

In [212]:
#meta信息
meta= {
        "user_info": "{role2_info}",
        "bot_info": "{role1_info}",
        "bot_name": "{role1_name}",
        "user_name": "{role2_name}"
    }

In [214]:
#生成角色人设并返回
profiles = generate_character_profiles(client, text)
role1_info,role2_info,role1_name,role2_name = profiles

In [216]:
#调用超拟人大模型根据给定的消息生成聊天的回复
reply_message = create_chat_completion(client, meta, messages)

In [218]:
print (reply_message.content)

（微笑着，眼中闪过一丝感慨） 我很高兴听到这些，艾米丽。是你让我相信，我的书不仅仅是商业工具，更是一种鼓舞人心的力量。



In [220]:
#将模型的回复添加到messages中
messages.append(reply_message.content)

In [222]:
print (messages)

[{'role': 'assistant', 'content': '（旁白：在那个充满书香的咖啡馆里，杰克和艾米丽坐在窗边，享受着温暖的阳光和浓郁的咖啡香。他们的对话在轻松愉快的氛围中展开。）艾米丽，你还记得你第一次站在演讲台上的样子吗？那时候你紧张得手都在抖。'}, {'role': 'user', 'content': '（轻笑着，眼中闪过一丝顽皮） 哦，杰克，你真是的。我那时候确实紧张，但你的鼓励让我迈出了那一步。'}, {'role': 'assistant', 'content': '（点头，眼中满是骄傲） 是的，你做到了。你的演讲总是那么鼓舞人心，就像你带领团队一样。'}, {'role': 'user', 'content': '（认真地） 我学到了很多，杰克。你的书教会了我如何用故事去影响别人，如何用文字去传递力量。'}, '（微笑着，眼中闪过一丝感慨） 我很高兴听到这些，艾米丽。是你让我相信，我的书不仅仅是商业工具，更是一种鼓舞人心的力量。\n']


In [224]:
#将对话记录保存到文件中
dialogue_filename = "dialogue.txt"
save_dialogue_to_file(messages, dialogue_filename)